# **ChatBot for CodeClause By Ameni CHAABOUNI**

**Import Libraries**


In [36]:
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Initialization**

In [37]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

**Loading Data**

In [38]:
data_file = open('/content/intents.json').read()
intents = json.loads(data_file)

In [39]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [40]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [41]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

**Lemmaztize and lower each word and remove duplicates**

In [42]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

**Sort classes**

In [43]:
classes = sorted(list(set(classes)))

**documents = combination between patterns and intents**

In [44]:
print (len(documents), "documents")

50 documents


**classes = intents**

In [45]:
print (len(classes), "classes", classes)

10 classes ['About CodeClause', 'Contact', 'goodbye', 'greeting', 'internships', 'internships duration', 'internships fields', 'offer_letter', 'options', 'thanks']


**words = all words, vocabulary**

In [46]:
print (len(words), "unique lemmatized words", words)

75 unique lemmatized words ["'s", ',', 'about', 'an', 'anyone', 'applied', 'apply', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'codeclause', 'concern', 'contact', 'could', 'day', 'do', 'duration', 'field', 'for', 'get', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'inernship', 'information', 'internship', 'is', 'later', 'letter', 'long', 'many', 'me', 'more', 'my', 'next', 'nice', 'not', 'of', 'offer', 'offered', 'provide', 'receieve', 'recieved', 'see', 'service', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'touch', 'what', 'when', 'where', 'which', 'will', 'with', 'you']


In [47]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


**Training**

In [48]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [49]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [50]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [51]:
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [52]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [53]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [54]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

Epoch 1/200
10/10 [==============================] - 1s 2ms/step - loss: 2.3279 - accuracy: 0.1200
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 2.2677 - accuracy: 0.1400
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1674 - accuracy: 0.3600
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0725 - accuracy: 0.3400
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0028 - accuracy: 0.3600
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 1.8234 - accuracy: 0.4200
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7928 - accuracy: 0.4200
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.7368 - accuracy: 0.4400
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.5543 - accuracy: 0.5400
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4407 - accuracy: 0.5400
Epoch 11/

In [55]:
print("model created")

model created


***************************************************************************************************************

**Import Libraries**

In [56]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import json
import random
from keras.models import load_model

**Loading Data**

In [57]:
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

**Creating Functions**

In [58]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [59]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [60]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [61]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [62]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [63]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

In [66]:
def start_chat():
     name = input("Enter Your Name: ")
     print("Welcome " + name + " to the Code Clause Bot Service! Let me know how can I help you?\n")
     while True:
         inp = str(input()).lower()
         if inp.lower()=="end":
             break
         if inp.lower()== '' or inp.lower()== '*':
             print('Please re-phrase your query!')
             print("-"*50)
         else:
             print(f"Code Clause: {chatbot_response(inp)}"+'\n')
             print("-"*50)

# start the chat bot
start_chat()

Enter Your Name: Ameni Chaabouni
Welcome Ameni Chaabouni to the Code Clause Bot Service! Let me know how can I help you?

What help you provide?
1/1 [==============================] - 0s 21ms/step
Code Clause: I can guide you if you have any query about Our Company CodeClause

--------------------------------------------------
what is codeclause?
1/1 [==============================] - 0s 17ms/step
Code Clause: CodeClause aspires to be the global sourcing choice of the world market and revolutionizes the way service processes function. 
To reach out to the common people across the globe and make Information Technology a tool for the “MASS” along with the tool for the “CLASS”. 
Creating innovative IT solutions and providing IT-enabled services to delight customers worldwide and build Relationships based on Trust, Values and Professionalism. CodeClause has industry-specific software expertise in Technology, Financial, Healthcare, Media, Manufacturing, and many other sectors. 
The company 

**Creating GUI**


In [ ]:
#We can use this code to run the ChatBot using GUI
#Creating GUI with tkinter
import tkinter
from tkinter import *
import sys
import os
if os.environ.get('DISPLAY','')=='':
  print('no DISPLAY found. Using: 0.0')
  os.environ.__setitem__('DISPLAY',':0.0')
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

# Thank You !